# Генерация больших данных для нагрузочного тестирования Hadoop

Этот ноутбук генерирует синтетический набор данных заданного размера и сохраняет его как таблицу Hive.

**Стратегия управления размером файлов (Native Spark AQE):**
- Мы включаем **Adaptive Query Execution (AQE)**.
- Задаем целевой размер партиции (`spark.sql.adaptive.advisoryPartitionSizeInBytes`) в **128 МБ**.
- Делаем `repartition` на заведомо **большое** количество частей (например, 1000).
- **Spark сам** объединяет (coalesce) эти мелкие части в файлы по ~128 МБ во время записи.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, rand, concat, expr, hex
import time

# --- ПАРАМЕТРЫ КОНФИГУРАЦИИ ---
NUM_ROWS = 1_000_000  # Желаемое количество строк
TABLE_NAME = "large_scale_table"
TARGET_FILE_SIZE_B = 128 * 1024 * 1024  # 128 MB в байтах
# ------------------------------

print(f"Конфигурация: {NUM_ROWS} строк, таблица '{TABLE_NAME}', цель по размеру файла: 128 MB")

Конфигурация: 1000000 строк, таблица 'large_scale_table', цель по размеру файла: 128 MB


In [2]:
spark = (
    SparkSession.builder
        .master("spark://spark-master:7077")
        .appName("LargeScaleDataGen")
        .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083")
        .enableHiveSupport()

        # --- ВКЛЮЧАЕМ AQE (Адаптивное выполнение) ---
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
        .config("spark.sql.adaptive.advisoryPartitionSizeInBytes", str(TARGET_FILE_SIZE_B))
        .config("spark.sql.parquet.block.size", str(TARGET_FILE_SIZE_B))

        .getOrCreate()
)

print("Spark Session создана с включенным AQE.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/18 21:06:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session создана с включенным AQE.


In [3]:
# Генерация данных
print("Формирование DataFrame...")

df = spark.range(0, NUM_ROWS).withColumnRenamed("id", "row_id")

df_with_data = df \
    .withColumn("val_int_1", (rand() * 100).cast("int")) \
    .withColumn("val_int_2", (rand() * 10000).cast("int")) \
    .withColumn("val_double_1", rand()) \
    .withColumn("val_double_2", rand() * 1000.0) \
    .withColumn("category_code", (rand() * 5).cast("int").cast("string")) \
    .withColumn("status", expr("elt(cast(rand()*3 as int) + 1, 'ACTIVE', 'INACTIVE', 'PENDING')")) \
    .withColumn("random_string_1", hex(expr("rand() * 100000").cast("long"))) \
    .withColumn("random_string_2", concat(lit("prefix_"), col("row_id"))) \
    .withColumn("created_date", expr("date_add(current_date(), -cast(rand()*365 as int))")) \
    .withColumn("description", lit("This is a test record for load testing Hadoop cluster capacity"))

print("Схема:")
df_with_data.printSchema()

Формирование DataFrame...
Схема:
root
 |-- row_id: long (nullable = false)
 |-- val_int_1: integer (nullable = true)
 |-- val_int_2: integer (nullable = true)
 |-- val_double_1: double (nullable = false)
 |-- val_double_2: double (nullable = false)
 |-- category_code: string (nullable = true)
 |-- status: string (nullable = true)
 |-- random_string_1: string (nullable = true)
 |-- random_string_2: string (nullable = false)
 |-- created_date: date (nullable = true)
 |-- description: string (nullable = false)



In [4]:
start_time = time.time()
print(f"Начинаем запись данных в таблицу {TABLE_NAME}...")

# СТРАТЕГИЯ AQE "OVERSHOOT":
# 1. Принудительно разбиваем данные на БОЛЬШОЕ количество мелких партиций.
#    Число 1000 берем с запасом (расчетно 100 млн строк это ~3 Гб, 
#    1000 партиций дадут ~3 Мб на партицию, что намного меньше 128 Мб).
# 2. AQE увидит, что партиции слишком маленькие, и объединит их до целевого размера (128 Мб).

OVER_PARTITION_COUNT = 1000

df_with_data \
    .repartition(OVER_PARTITION_COUNT) \
    .write \
    .mode("overwrite") \
    .format("parquet") \
    .saveAsTable(f"default.{TABLE_NAME}")

end_time = time.time()
duration = end_time - start_time

print(f"✓ Готово! Записано {NUM_ROWS} строк.")
print(f"Время выполнения: {duration:.2f} секунд")

Начинаем запись данных в таблицу large_scale_table...


26/01/18 21:09:34 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


✓ Готово! Записано 1000000 строк.
Время выполнения: 174.59 секунд


In [5]:
# Проверка результата
spark.sql(f"SELECT * FROM default.{TABLE_NAME} LIMIT 5").show(truncate=False)

count = spark.sql(f"SELECT count(*) FROM default.{TABLE_NAME}").collect()[0][0]
print(f"Проверка количества строк: {count}")

+------+---------+---------+--------------------+------------------+-------------+--------+---------------+---------------+------------+--------------------------------------------------------------+
|row_id|val_int_1|val_int_2|val_double_1        |val_double_2      |category_code|status  |random_string_1|random_string_2|created_date|description                                                   |
+------+---------+---------+--------------------+------------------+-------------+--------+---------------+---------------+------------+--------------------------------------------------------------+
|200644|47       |994      |0.024814132855865978|230.371993448805  |1            |INACTIVE|11C2D          |prefix_200644  |2025-12-10  |This is a test record for load testing Hadoop cluster capacity|
|497917|56       |6162     |0.12605512751903447 |714.1828741755163 |4            |PENDING |2F0C           |prefix_497917  |2025-08-19  |This is a test record for load testing Hadoop cluster capacity|


[Stage 4:=================================================>       (28 + 2) / 32]

Проверка количества строк: 1000000


In [6]:
spark.stop()